In [0]:
import tensorflow as tf

#mnist데이터셋을 다운로드 한다.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

In [0]:
# CNN모델을 정의
def build_CNN_classifier(x):
  #mnist 3차원 형태로 reshape 28*28*1
  x_image = tf.reshape(x, [-1,28,28,1])

  #1st conv2d
  # 5x5 32 filter(channel)
  # 28x28x1 -> 28x28x32
  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,1,32], stddev=5e-2))
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1,strides=[1,1,1,1],padding='SAME')+b_conv1)
  #1st max pooling
  #입력 이미지를 1/2로 다운샘플링
  # 28x28x32 -> 14x14x32
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

  #2nd conv2d
  # 5x5 64 filter(channel)
  # 14x14x32 -> 14x14x64
  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,32,64], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2,strides=[1,1,1,1],padding='SAME')+b_conv2)
  #2nd max pooling
  #입력 이미지를 1/2로 다운샘플링
  # 14x14x64 -> 7x7x64
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

  #Fully connected Layer
  #7x7x64(3136) -> 1024
  # 7x7크기를 가진 64개의 activation map을 1024개의 feature로 변환
  W_fc1 = tf.Variable(tf.truncated_normal(shape=[7*7*64,1024], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
  h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
  h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

  #output Layer
  #1024개의 특징들을 10개의 클래스로 변환한다.
  W_output = tf.Variable(tf.truncated_normal(shape=[1024, 10], stddev=5e-2))
  b_output = tf.Variable(tf.constant(0.1, shape=[10]))
  logits = tf.matmul(h_fc1, W_output)+b_output
  y_pred = tf.nn.softmax(logits)

  return y_pred, logits

In [0]:
# placeholder정의
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

# Parameter
learningRate = 1e-4
epochs = 10000
batchSize = 50
displayStep = 100

y_pred, logits = build_CNN_classifier(x)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y))
train_step = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(loss)

#학습이 끝나면 모델의 정확도를 확인한다.
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  #epoch만큼 최적화를 수행한다.
  for i in range(epochs):
    batch = mnist.train.next_batch(batchSize)
    if i % displayStep ==0:
      train_acuuracy = accuracy.eval(feed_dict={x:batch[0], y:batch[1]})
      print("Epoch: %d, train_acuuracy: %f" %((i+1), train_acuuracy))
    sess.run([train_step], feed_dict={x:batch[0], y:batch[1]}) 

  print("Accuracy: %f" %(accuracy.eval(feed_dict={x:mnist.test.images, y: mnist.test.labels}))) 